In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import scipy
from scipy import linalg
from scipy.special import logsumexp

Populating the interactive namespace from numpy and matplotlib


$\def\m#1{\mathbf{#1}}$
$\def\mm#1{\boldsymbol{#1}}$
$\def\mb#1{\mathbb{#1}}$
$\def\c#1{\mathcal{#1}}$
$\def\mr#1{\mathrm{#1}}$
$\newenvironment{rmat}{\left[\begin{array}{rrrrrrrrrrrrr}}{\end{array}\right]}$
$\newcommand\brm{\begin{rmat}}$
$\newcommand\erm{\end{rmat}}$
$\newenvironment{cmat}{\left[\begin{array}{ccccccccc}}{\end{array}\right]}$
$\newcommand\bcm{\begin{cmat}}$
$\newcommand\ecm{\end{cmat}}$
# Homework 5
## Homework guideline
- The deadline is Dec 5th 11:59pm. Submission after the deadline will not be graded.


- Submit your work(your reasoning and your code) as a SINGLE .ipynb document. Please rename the document as _HW1_YOURNAME.ipynb_ (for example, _HW1_FELIX.ipynb_). You are responsible for checking that you have correctly submitted the correct document. If your code cannot run, you may receive NO point.

- Please justify all short answers with a brief explanation. If you use latex command in the markdown, **2 points** bonus will be awarded.   



- You only use the Python packages included in the following cell. You are not allowed to use other advanced package or modules unless you are permitted to.

- In your final submission include the plots produced by the unedited code as presented below, as well as any additional plots produced after editing the code during the course of a problem. You may find it necessary to copy/paste relevant code into additional cells to accomplish this.

- Feel free to use the lecture notes and other resources. But you
must understand, write, and hand in your own answers. In addition, you must write and submit
your own code in the programming part of the assignment (we may run your code).
If you copy someone else homework solution, both of you may receive ZERO point.


- Colab is preferred. However, if you use Anaconda, please download the .mat file locally and save it to the same folder as this homework file.





---



---


# Q1: EM algorithm for mixture models (20pt)
Mixture models are statistical models of the form
\begin{align}
\large p(\m x;\mm \theta)= \sum_{k=1}^K \pi_k p_k(\m x; \mm \theta_k )
\end{align}
where each $p_k(\m x; \mm \theta_k )$ is itself a statistical model parameterised by $\mm \theta_k$ and $\pi_k\ge 0$ are mixture
weights that sum to one. The parameters $\mm \theta$ of the mixture model consist of the parameters $\mm\theta_k$ of each mixture component and the mixture weights $\pi_k$, i.e., $\mm \theta = (\mm \theta_1,\dots, \mm \theta_K, \pi_1, \dots, \pi_K)$. An
example is a mixture of Gaussians where each $p(\m x;\mm \theta)$ is a Gaussian with parameters given by
the mean vector $\mm \mu_k$ and a covariance matrix $\mm \Sigma_k$.

The mixture model  can be considered to be the marginal distribution of a latent variable
model $p(\m x, h; \mm \theta)$ where $h$ is an unobserved variable that takes on values $1, \dots, K$ and $p(h=k)=\pi_k$. Defining $p(\m x|h=k; \mm \theta)=p_k(\m x; \mm \theta_k)$, the latent variable model thus is
\begin{align}
p(\m x, h=k;\mm \theta) = p(\m x| h=k; \mm \theta) p(h=k)= \pi_k p_k(\m x; \mm \theta_k)
\end{align}

In particular note that marginalising out $h$ gives $p(\m x; \mm \theta)$.


---


## Q1.1 (2pt)
Verify that the latent variable model can be written as
\begin{align}
 p(\m x, h; \mm \theta) = \Pi_{k=1}^K[\pi_k p_k(\m x; \mm \theta_k) ]^{\mathbb{1}(h=k)}
\end{align}
where $h$ takes values in $1, \dots, K$.





# Your Solution:
$$
\begin{align*}
\Pi_{k=1}^K[\pi_k P_k(\m{x} | \mm{\theta}_k )]^{1_{h=k}} &= \Pi_{k=1}^K[ P(\m x| h=k; \mm \theta) P(h=k)]^{1_{h=k}}\\
&= \Pi_{k=1}^K[ P(\m x, h=k; \mm \theta)]^{1_{h=k}}\\
\text{Since we take } h = k, \text{ we get } &= \Pi_{k=1}^K P(\m x, h=k; \mm \theta)\\
&= P(\m x, h | \mm \theta)
\end{align*}
$$


---




## Q1.2 (8pt)
Since the mixture model  can be seen as the marginal of a latent-variable model, we
can use the expectation maximisation (EM) algorithm to estimate the parameters $\mm \theta$.

For a general model $p(\mathcal{D}, \m h; \mm \theta)$ where $\mathcal{D}$ are the observed data and $\m h$ the corresponding unobserved variables, the EM algorithm iterates between computing the expected complete-data log-likelihood $J^{l}(\mm \theta)$ are maximising it with respect to $\mm \theta$:


- E-step at iteration $l$: $$J^{l}(\mm \theta)=
\mathbb{E}_{p(\m h|\mathcal{D}; \mm \theta^l)}[\log p(\mathcal{D}, \m h; \mm \theta)]$$

- M-step at iteration $l$:
$$\mm \theta^{l+1}=\arg\max_{\mm \theta} J^l(\mm \theta) $$

Here $\mm \theta^l$ is the value of $\mm \theta$ in the $l$-th iteration. When solving the optimisation problem, we
also need to take into account constraints on the parameters, e.g. that the $\pi_k$ correspond to a probability mass function.

Assume that the data $\mathcal{D}$ consists of $n$ i.i.d. data points $\m x_i$, that each $\m x_i$ has associated with it a scalar unobserved variable $h_i$, and that the tuples $(\m x_i, h_i)$ are all i.i.d.

Show that for the latent variable model, $J^l(\mm \theta)$ equals
\begin{align}
J^l(\mm \theta) &= \sum_{i=1}^n \sum_{k=1}^K w_{ik}^l \log[\pi_k p_k(\m x_i; \mm \theta_k)]\\
w_{ik}^l &= \frac{\pi_k^l p_k(\m x_i; \mm \theta_k^l)}{\sum_{k=1}^K \pi_k^l p_k(\m x_i; \mm \theta_k^l)}
\end{align}

Note that the $w_{ik}^l$ are defined in terms of parameters $\pi_k^l$ and $\mm \theta_k^l$ from iteration $l$. They are equal to the conditional probabilities $p(h=k |\m x_i; \mm \theta^l)$, i.e., the probability that $\m x_i$ has been sampled from component $p_k(\m x_i; \mm \theta_k^l)$.

# Your solution:

$$
\begin{align*}
J^{l}(\mm \theta) &= \mathbb{E}_{P(\m h|\mathcal{D}; \mm \theta^l)}[\log P(\mathcal{D}, \m h; \mm \theta)]\\
&= \mathbb{E}_{P(\m h|\mathcal{D}; \mm \theta^l)} \log \Pi_{i=1}^n P(\m{x}_i, \m h; \mm \theta)\\
&= \mathbb{E}_{P(\m h|\mathcal{D}; \mm \theta^l)}  \sum_{i=1}^n \log P(\m{x}_i, \m h; \mm \theta)\\
&= \sum_{i=1}^n \mathbb{E}_{P(\m h|\m x_i; \mm \theta^l)} \log P(\m{x}_i, \m h; \mm \theta)\\
\text{From 1.1: } &= \sum_{i=1}^n ( \sum_{k=1}^K [ P(\m h = k| \m x_i; \mm \theta^l) * \log \Pi_{k=1}^K[\pi_k P_k(\m{x}_i | \mm{\theta}_k )]^{1_{h=k}} ] )\\
&= \sum_{i=1}^n (\sum_{k=1}^K [ P(\m h = k| \m x_i; \mm \theta^l) * \sum_{k=1}^K 1_{h=k}\log\pi_k P_k(\m{x}_i | \mm{\theta}_k ) ] )\\
\text{where when } h\neq k, 1_{h=k} = 0, \text{ so }  &= \sum_{i=1}^n \sum_{k=1}^K P(\m h = k| \m x_i; \mm \theta^l) \log \pi_k P_k(\m{x}_i; \mm \theta_k)\\
\text{Bayes' Product Rule }&= \sum_{i=1}^n \sum_{k=1}^K  \frac{P(\m h=k, \m x_i | \mm\theta)}{P(\m x_i | \mm \theta)}  \log \pi_k P_k(\m{x}_i; \mm \theta_k)\\
&= \sum_{i=1}^n \sum_{k=1}^K  \frac{\pi_k^l P_k(\m x_i; \mm \theta_k^l)}{\sum_{k=1}^K \pi_k^l P_k(\m x_i; \mm \theta_k^l)}  \log \pi_k P_k(\m{x}_i; \mm \theta_k)\\
&= \sum_{i=1}^n \sum_{k=1}^K  w_{ik}^l  \log \pi_k P_k(\m{x}_i; \mm \theta_k)
\end{align*}
$$



---
## Q1.3: (5pt)
Assume that the different mixture components $p_k(\m x; \mm \theta_k), k=1,\dots, K$ do not share any
parameters. Show that the updated parameter values $\mm \theta_k^{l+1}$ are given by weighted maximum
likelihood estimates.


# Your solution:

$$
\begin{align*}
\mm\theta^{l+1} = \{\pi^{l+1}, \mm\mu^{l+1}, \mm\Sigma^{l+1} \} &= \arg\max_{\mm \theta} J^l(\mm \theta) \\
&= \arg\max_{\mm \theta} \sum_{i=1}^n \sum_{k=1}^K  w_{ik}^l  \log (\pi_k P_k(\m{x}_i; \mm \theta) )\\
\text{For a specific } k, \text{ say } a, \text{ we get } &= \arg\max_{\mm \theta} \sum_{i=1}^n \sum_{k=1}^K  w_{ik}^l  \log ( \pi_k P_k(\m{x}_i; \mm \theta) ) 1_{k=a}\\
&= \arg\max_{\mm \theta_a} \sum_{i=1}^n w_{ia}^l  \log ( \pi_a P_a(\m{x}_i; \mm \theta_a) )\\
\text{Where } \mm\theta_a = \{\pi_a, \mm\mu_a, \mm\Sigma_a \}, \text{ so, } &=\arg\max_{\{\pi_a, \mm\mu_a, \mm\Sigma_a \}} \sum_{i=1}^n w_{ia}^l  \log ( \pi_a P_a(\m{x}_i; \{\pi_a, \mm\mu_a, \mm\Sigma_a \}) )\\
&=\arg\max_{\pi_a, \mm\mu_a, \mm\Sigma_a } \sum_{i=1}^n w_{ia}^l  \log ( \pi_a P_a(\m{x}_i; \mm\mu_a, \mm\Sigma_a ) )\\
&=\arg\max_{\pi_a } \sum_{i=1}^n w_{ia}^l  \log ( \pi_a ) + \arg\max_{ \mm\mu_a, \mm\Sigma_a } \sum_{i=1}^n w_{ia}^l \log(P_a(\m{x}_i; \mm\mu_a, \mm\Sigma_a ) )\\
\end{align*}
$$

We can take the derivative of this with respect to the parameters $\pi_a, \mm\mu_a, \mm\Sigma_a$ and set equal to 0 to get the weighted maximum likelihood estimates of each parameter, which will be $\{\hat{\pi}_a^{l+1}, \hat{\mm{\mu}}_a^{l+1}, \hat{\mm\Sigma}_a^{l+1} \} = \mm\theta_a^{l+1}$. The values are the following:

$$
\begin{align*}
& \pi_a^{l+1}=\frac{\sum_{i=1}^n w_{ia}^l}{n}\\
\\
& \mm\mu_k^{l+1} = \frac{\sum_{i=1}^n w_{ia}^l  \m x_i}{\sum_{i=1}^n w_{ia}^l}\\
\\
& \mm\Sigma_k^{l+1} = \frac{\sum_{i=1}^n w_{ia}^l  \left[\m x_i - \mm \mu_k^{l+1}\right]\left[\m x_i - \mm \mu_k^{l+1} \right]^T}{\sum_{i=1}^n w_{ia}^l}\\
\end{align*}
$$



---
## Q1.4: (5pt)
Show that maximising $J^l(\mm \theta)$ with respect to the mixture weights $\pi_k$ gives the update rule
\begin{align}
\pi_k^{l+1} = \frac{1}{n}\sum_{i=1}^n w_{ik}^l
\end{align}
 Summarise the EM-algorithm to learn the parameters $\mm \theta$ of the mixture model from i.i.d. data $\m x_1, \dots, \m x_n$.



# Your solution:

From 1.3, we can see that the only portion of $J^l(\mm \theta)$ we need to maximize when considering $\pi_a$ is the portion that considers $\pi_a$, which is $\sum_{i=1}^n w_{ia}^l  \log ( \pi_a )$. We also must consider the constraint $\sum_{k=1}^K \pi_k = 1$. We do this by using the Lagrangian (where $\nu$ is the Lagrangian multiplier):

$$
\begin{align*}
L(\pi) &= \sum_{i=1}^n \sum_{k=1}^K  w_{ik}^l  \log (\pi_k) + \nu \sum_{k=1}^k \pi_k - 1\\
\frac{\delta}{\delta \pi_a} J^l(\mm \theta) = \frac{d}{d\pi_a} L(\pi) &= \frac{d}{d\pi_a} \sum_{i=1}^n \sum_{k=1}^K  w_{ik}^l  \log (\pi_k) + \nu (\sum_{k=1}^K \pi_k - 1 ) = 0 \\
&\Rightarrow \sum_{i=1}^n \frac{w_{ia}^l}{\pi_a} + \nu =0\\
\Rightarrow \pi_a^{l+1} &= \frac{\sum_{i=1}^n w_{ia}^l}{-\nu}\\
&= \frac{\sum_{i=1}^n w_{ia}^l}{\sum_{i=1}^n \sum_{k=1}^K w_{ik}^l}\\
&= \frac{\sum_{i=1}^n w_{ia}^l}{\sum_{i=1}^n 1}\\
&= \frac{\sum_{i=1}^n w_{ia}^l}{n}.\\
\\
\end{align*}
$$

The EM Algorithm takes a training set $\mathcal{D}$ of observed data and some discrete hidden variables and uses the various formulas mentioned above to systematically gather information about a parameter $\mm \theta = \{\pi, \mm \mu, \mm \Sigma\}$ at some step $l$, i.e. we calculate an initial $\mm \theta^0$ and use it to calculate the next $\mm \theta^1$ and so on. This is done via two steps: an Expectation step, which gives the "clustering" or "latent" variable, and a Maximization step, which updates the parameter $\mm \theta^l$ to the next state $\mm \theta^{l+1}$ using the latent variables calculated in the first step. We do this step until we've reached a predetermined stopping criteron, usually based around the accuracy of our clusters.



---



---

# Q2: Theory in Hidden Markov Model (30pt)
Consider the hidden Markov model specified by the following graph
<img src="https://github.com/yexf308/AppliedStochasticProcess/blob/main/HW/HW4/DAG.png?raw=true" width="800" />





---
## Q2.1: Forward filtering backward sampling
We assume that have already run the alpha-recursion (filtering) and can compute $p(h_t|v_{1:t})$
for all $t$. The goal is now to generate samples $p(h_1,\dots h_n|v_{1:n}),$ i.e. entire trajectories $(h_1,\dots, h_n)$ from the posterior. Note that this is not the same as sampling from the $n$ filtering distributions $p(h_t|v_{1:t})$. Moreover, compared to the Viterbi algorithm, the sampling
approach generates samples from the full posterior rather than just returning the most probable state and its corresponding probability.

### Q2.1.1: (5pt)
Since $p(h_1,\dots, h_n|v_{1:n})$ is a first-order Markov chain, it suffices to determine $p(h_{t-1}|h_t; v_{1:n})$,
the probability mass function for $h_{t-1}$ given $h_t$ and all the data $v_{1:n}$. Show that
\begin{align}
p(h_{t-1}, h_t| v_{1:n}) \propto \alpha(h_{t-1})\beta(h_t)p(h_t| h_{t-1})p(v_t|h_t)
\end{align}


# Your Solution:

$$
\begin{align*}
P(h_{t-1}, h_t | v_{1:n}) &= \frac{P(h_{t-1}, h_t, v_{1:n})}{P(v_{1:n})}\\
&= \frac{P(h_{t-1}, v_{1:t-1})P(v_{t:n}, h_t | h_{t-1})}{P(v_{1:n})}\\
&= \frac{\alpha(h_{t-1})P(h_t | h_{t-1})P(v_{t:n}|h_t)}{P(v_{1:n})}\\
&= \frac{\alpha(h_{t-1})P(h_t | h_{t-1})P(v_t | h_t)P(v_{t+1:n} | h_t)}{P(v_{1:n})}\\
&= \frac{\alpha(h_{t-1})P(h_t | h_{t-1})P(v_t | h_t)\beta(h_t)}{P(v_{1:n})}\\
&\propto \alpha(h_{t-1})\beta(h_t)P(h_t| h_{t-1})P(v_t|h_t)
\end{align*}
$$

### Q2.1.2: (10pt)
Show that
\begin{align}
p(h_{t-1}| h_t, v_{1:n}) =\frac{\alpha(h_{t-1})}{\alpha(h_t)}p(h_t| h_{t-1})p(v_t|h_t)
\end{align}

# Your Solution:

$$
\begin{align*}
P(h_{t-1}| h_t, v_{1:n}) &= \frac{P(h_{t-1}, h_t, v_{1:n})}{P(h_t, v_{1:n})}\\
\\
\text{From the numerator in 2.1: }&= \frac{\alpha(h_{t-1})\beta(h_t)P(h_t| h_{t-1})P(v_t|h_t)}{P(h_t, v_{1:n})}\\
\\
&= \frac{\beta(h_t)}{P(h_t, v_{1:t}, v_{t+1:n})}*\alpha(h_{t-1})P(h_t| h_{t-1})P(v_t|h_t)\\
\\
\text{Using } P(v_{t+1:n} | h_t, v_{1:t}) &= \frac{P(h_t, v_{1:t}, v_{t+1:n})}{P(h_t, v_{1:t})},
\\
\\
\text{We have } &= \frac{\beta(h_t)}{P(h_t, v_{1:t})P(v_{t+1:n} | h_t, v_{1:t})}*\alpha(h_{t-1})P(h_t| h_{t-1})P(v_t|h_t)\\
\\
&= \frac{\beta(h_t)}{P(h_t, v_{1:t})P(v_{t+1:n} | h_t)}*\alpha(h_{t-1})P(h_t| h_{t-1})P(v_t|h_t)\\
\\
&= \frac{\beta(h_t)}{\alpha(h_t)\beta(h_t)}*\alpha(h_{t-1})P(h_t| h_{t-1})P(v_t|h_t)\\
\\
&=\frac{\alpha(h_{t-1})}{\alpha(h_t)}P(h_t| h_{t-1})P(v_t|h_t).
\end{align*}
$$




---




## Q2.2: Implement forward filtering backward sampling (15pt)
We now consider the sampling of latent states $h$ given an observation $v$. Firstly, it is important to differentiate sampling from Viterbi-backtracking:
* Viterbi-backtracking: $h_{1:n}^* = \text{argmax } p(h_{1:n} | v_{1:n})$
    * There is only one $h_{1:n}^*$ that has the largest probability (under the assumption of a unique global maximum).
* Sampling: $h_{1:n} \sim p(h_{1:n} | v_{1:n})$
    * In sampling we attempt to generate several latent sequences $h_{1:n}$, not just the most probable.
    * Note that the sequence with $h_{1:n}^*$ still has the largest probability to get sampled.


It is also helpful to view sampling as a random function (as opposed to a deterministic function):
* Viterbi-backtracking is a deterministic function since it always gives the same output.
* Sampling is a random function since each time it may give us different outputs.

Sampling from the posterior $p(h_{1:n} | v_{1:n})$ can be done with backward-sampling:
\begin{align}
    h_n &\sim p(h_n | v_{1:n}) \\
    h_{t-1} &\sim p(h_{t-1} | h_t, v_{1:n}) \qquad \text{for }t\le n
\end{align}

In the last problem, we showed that
\begin{align}
p(h_{t-1}| h_t, v_{1:n}) =\frac{\alpha(h_{t-1})}{\alpha(h_t)}p(h_t| h_{t-1})p(v_t|h_t)
\end{align}
Since the probabilities above are expressed as a function of the transition, emission, and the alpha variables, we will use the forward algorithm to obtain them.

## Algorithm
We thus obtain the following algorithm to generate samples from $p(h_1, \dots, h_n | v_{1:n})$:

- Run the filtering to determine all $\alpha(h_t)$ forward in time for $t=1, \dots, n$.

- Sample $h_n$ from $p(h_n|v_{1:n})\propto\alpha(h_n)$.

-  Go backwards in time using
\begin{align}
p(h_{t-1}| h_t, v_{1:n}) =\frac{\alpha(h_{t-1})}{\alpha(h_t)}p(h_t| h_{t-1})p(v_t|h_t)
\end{align}
to generate samples $h_{t-1}| h_t, v_{1:n}$ for $t=n,\dots, 2$.

You will implement this algorithm in the following function.
The computation of the equations above are performed in the log domain, hence

\begin{align*}
\log p(h_{t - 1}| h_t, v_{1:T}) &=  \log \alpha(h_{t-1}) + \log p(h_t | h_{t-1}) + \log p(v_t | h_t) - \log \alpha(h_t)
\end{align*}



In [ ]:
def forward(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T

    Returns:
        log_alpha: a matrix of shape T * N, log_alpha[t, n] denote the sum of all possible latent state sequences ending at step t, state n
        log_Z: a float number
    """
    T = len(v)
    N = log_emission.shape[1]

    log_alpha = np.zeros((T, N))
    log_alpha[0] = log_emission[v[0]] + log_initial

    for t in range(1, T):
        log_emission_t = log_emission[v[t]]
        log_alpha[t] = logsumexp(log_alpha[t - 1].reshape(N, 1) +
                                 log_transition +
                                 log_emission_t.reshape(1, N),
                                 axis=0)


    log_Z = logsumexp(log_alpha[T - 1], axis=0)
    return log_alpha , log_Z

In [64]:
def sampling(log_initial, log_transition, log_emission, v):
    """
    Args:
        initial: a vector of length N
        transition: a matrix of shape N * N
        emission: a matrix of shape V * N
        v: observed sequence, a vector of length T

    Returns:
        h: sampled sequence, a vector of length T
    """

    # Filtering to get the log alpha matrix
    log_alpha, log_Z = forward(log_initial, log_transition, log_emission, v)
    N = log_emission.shape[1]
    n = len(log_initial)
    index_matrix = []
    for i in range(n):
      index_matrix.append(i)

    # Initializing
    h = np.zeros(n, dtype=int)
    t = n

    while t != 0:
      t-=1
      # Initial case for h
      if t == n-1:
        prob = np.exp(log_alpha[t] - log_Z)
        h[t] = np.random.choice(index_matrix, p=prob)

      # Iterative case for h
      else:
        alpha = log_alpha[t] - log_alpha[t+1]
        prob = np.exp(alpha +
                      log_transition[:,h[t+1]] +
                      log_emission[v[t+1]][h[t+1]])
        # For some reason, this probability vector would not give me sum to 1,
        # so there is an error with respect to this. I'm not sure where my
        # error is.
        prob = prob/sum(prob)
        h[t] = np.random.choice(index_matrix, p=prob)

    return h

## Verify the algorthm with examples
### Test case 1
we call the sampling function to see its output. Try running it multiple times to see that the outputs can be different.

In [123]:
log_initial = np.log(np.array([0.3, 0.7]))
log_transition = np.log(np.array([[0.2, 0.8],
                                  [0.6, 0.4]]))
log_emission = np.log(np.array([[0.6, 0.7],
                                [0.4, 0.3]]))
v = [0, 0]

print(sampling(log_initial, log_transition, log_emission, v))


[1 1]


### Test case 2
We verify our implementation by the law of large numbers: if we call the sampling function many times, then the frequency of each sampled latent sequence will be approximately equal to their probability.

Note that the frequency of the latent sequence [1, 0] returned from the simulation should close to 0.4045, i.e., approximately same as the true probability returned by the Viterbi algorithm. You may increase the number `N=100000`, which should return an even closer result.

In [125]:
h00 = 0
h01 = 0
h10 = 0
h11 = 0

N = 100000
for _ in range(N):
    h = sampling(log_initial, log_transition, log_emission, v)
    if h[0] == 0:
      if h[1] == 0:
        h00+=1
      else:
        h01+=1
    else:
      if h[1] == 0:
        h10+=1
      else:
        h11+=1

print('h00 prob is', h00/N)
print('h01 prob is', h01/N)
print('h10 prob is', h10/N)
print('h11 prob is', h11/N)

h00 prob is 0.05915
h01 prob is 0.25398
h10 prob is 0.39538
h11 prob is 0.29149






---



---

# Q5: (optional) Correction to your previous homework question
You may pick any questions in homework 1-4 that didn't perform well, and now you have the chance to correct your mistakes. If you successfully correct your mistakes, your previous grade will be replaced by the current score.

**State Your question that you want to correct:**



---


## HW3 Q1.2 (***) (10pt)
Suppose two distinct states $i, j$ of a Markov chain satisfy
$$ P(T_j< T_i|X_0=i)= P(T_i < T_j | X_0=j)$$
where $T_j=\inf\{n\ge 1: X_n=j\}$, which is the first return time to state $j$. Show that, if $X_0 = i$ the expected number of visits to $j$ prior to re-visiting $i$ is one, i.e.,
$$\mb{E}_i[\sum_{n=1}^\infty \m{I}(X_n=j) \m{I}(n\le T_i)]=1 $$


# Your Solution:

Firstly, let's let $P(T_j< T_i|X_0=i) = P(T_i < T_j | X_0=j) = p.$ Then,

$$
\begin{align*}
\mb{E}_i[\sum_{n=1}^\infty \m{I}(X_n=j) \m{I}(n\le T_i)] &= \sum_{n=1}^\infty
\mb{E}_i [ \m{I}(X_n=j) \m{I}(n\le T_i) ] \\
&= \sum_{n=1}^\infty P( X_n = j, n\le T_i | X_0 = i)
\end{align*}
$$

We consider cases intuitively. Suppose we reach state $j$ once before returning to state $i.$ Then, we would have probability $p$ to do so. This is our $n=1$ case.

Now, suppose once we are at state $j$, we stay at state $j$, or continuously return to state $j$ before returning to state $i$. This probability is, when starting at state $i$, $p*(1-p)^k$, where $k$ is the number of times we return to state $j$ after reaching it. We say $(1-p)$ since the probability to return to state $i$ from $j$ is equal to going from state $i$ to $j$. So, returning to $j$ from $j$ has probability $1-p$.

If we only return to state $j$ once, we have visited state $j$ twice, i.e $n=2$, our probability is $p*(1-p)$. If we visit state $j$ some number of times, say $k+1$ times, we have probability $p*(1-p)^k.$ Then, we can equate the summation above as the following:

$$
\begin{align*}
\sum_{n=1}^\infty P( X_n = j, n\le T_i | X_0 = i) &= p+p*(1-p)+p*(1-p)^2+...+p(1-p)^k+...\\
&= p*(1+(1-p)+(1-p)^2+...+(1-p)^k+...)\\
&= p*\sum_{n=0}^\infty (1-p)^n\\
&= p*\frac{1}{1-(1-p)}\\
&= p*\frac{1}{p}\\
&= 1
\end{align*}
$$



---


## Q1.3: Reversiblility (*) (10pt)

A Markov chain on $S=\{0,1,\dots, N\}$ has transition probabilities $P(0,0) = 1-\lambda_0, P(i,i+1)=\lambda_i, P(i+1, i) =\mu_{i+1}$ for $i=0,\dots, N-1$ and $P(N,N)=1-\mu_N$. Show that the process
is reversible in equilibrium.  

# Your Solution:

We consider the transitiion matrix outlined in the problem above as the following:

$$
\bcm 1-\lambda_0 & \lambda_0 & 0 & 0 & ... & 0 & 0 & 0 & 0 \\
     \mu_1 & 0 & \lambda_1 & 0 & ... & 0 & 0 & 0 & 0 \\
     0 & \mu_2 & 0 & \lambda_2 & ... & 0 & 0 & 0 & 0 \\
     ... & ... & ... & ... & ... & ... & ... & ... & ... \\
     0 & 0 & 0 & 0 & ... & \mu_{n-2} & 0 & \lambda_{n-2} & 0 \\
     0 & 0 & 0 & 0 & ... & 0 & \mu_{n-1} & 0 & \lambda_{n-1}\\  
     0 & 0 & 0 & 0 & ... & 0 & 0 & \mu_n & 1-\mu_n\ecm.
$$

We now consider each $\pi_i$. Note that $\mu_i+\lambda_i = 1 \;\forall i\in\{1,2,...,n-1\}$, so $\lambda_i = 1-\mu_i$.

First, consider $i=0$. Then,

$$
\begin{align*}
\pi_0 &= \pi_0(1-\lambda_0) + \pi_1\mu_1\\
&= \pi_0 - \pi_0\lambda_0 + \pi_1\mu_1\\
\Rightarrow 0 &= -\pi_0\lambda_0 + \pi_1\mu_1\\
\Rightarrow \pi_0\lambda_0 &= \pi_1\mu_1.
\end{align*}
$$

Now, for $i=1$,

$$
\begin{align*}
\pi_1 &= \pi_0\lambda_0 + \pi_2\mu_2\\
&= \pi_1\mu_1 + \pi_2\mu_2\\
\Rightarrow \pi_1(1-\mu_1) &= \pi_2\mu_2\\
\Rightarrow \pi_1\lambda_1 & = \pi_2\mu_2.
\end{align*}
$$

With the intent to prove via induction, assume for some $1 \lt k \lt n,\; \pi_{k-1}\lambda_{k-1}=\pi_k\mu_k$. Then, we have the following:

$$
\begin{align*}
\pi_k &= \pi_{k-1}\lambda_{k-1} + \pi_{k+1}\mu_{k+1}\\
&= \pi_k\mu_k + \pi_{k+1}\mu_{k+1}\\
\Rightarrow \pi_k(1-\mu_k) &= \pi_{k+1}\mu_{k+1}\\
\Rightarrow \pi_k\lambda_k & = \pi_{k+1}\mu_{k+1}.
\end{align*}
$$

Finally, for $i=n$, while proven via the induction step, confirms our findings:

$$
\begin{align*}
\pi_n &= \pi_{n-1}\lambda_{n-1} + \pi_n(1-\mu_n)\\
&= \pi_{n-1}\lambda_{n-1} + \pi_n - \pi_n\mu_n\\
\Rightarrow 0 &= \pi_{n-1}\lambda_{n-1} - \pi_n\mu_n\\
\Rightarrow \pi_{n-1}\lambda_{n-1} &= \pi_n\mu_n.
\end{align*}
$$

So, for all $i \in S \setminus \{0\}, \pi_{i-1}\lambda_{i-1} = \pi_i\mu_i$ (excluding $i=0$ as $-1\notin S$, but when $i=1, \pi_1\mu_1 = \pi_{1-1}\lambda_{1-1} = \pi_0 \lambda_0$, so this set is inclusive of all "$\pi_i$ equalities").

Now, consider any two $i,j \in S$.

If $i=j, P_{i,j} = P_{i,i} = P_{j,i}$ for all $i,j$ and clearly $\pi_i = \pi_j$, so we have the desired result of $\pi_i P_{i,j} = \pi_j P_{j,i}$ when $i=j$.

If $i\neq j, |i-j|>1$, then $P_{i,j} = P_{j,i} = 0$, so trivially, $\pi_i P_{i,j} = \pi_j P_{j,i} = 0$.

Finally, if $i\neq j, |i-j|=1$, say $i-j = 1 \Rightarrow j=i-1$, we have:

$$
\begin{align*}
\pi_i P_{i,j} &= \pi_i P_{i,i-1}\\
&= \pi_i \mu_i\\
&= \pi_{i-1} \lambda_{i-1}\\
&= \pi_{i-1} P_{i-1,i}\\
&= \pi_j P_{j,i}.
\end{align*}
$$

Note that $j-i=1 \Rightarrow i=j-1$, so we have the same result:

$$
\begin{align*}
\pi_j P_{j,i} &= \pi_j P_{j,j-1}\\
&= \pi_j \mu_j\\
&= \pi_{j-1} \lambda_{j-1}\\
&= \pi_{j-1} P_{j-1,j}\\
&= \pi_i P_{i,j}.
\end{align*}
$$

So, for all $i,j \in S$, we have $\pi_i P_{i,j} = \pi_j P_{j,i}$, therefore, the Markov Chain will be reversible given some $\pi$, however this $\pi$ may be trivial. We prove that $\sum_{i=1}^n \pi_i = 1$ using the system of equations:

$$
\begin{align*}
\{\pi_0 P_{0,1} - \pi_1 P_{1,0} = 0,...,\pi_{n-1} P_{n-1, n} - \pi_n P_{n, n-1} &=0, \sum_{i=1}^n \pi_i - 1 = 0\}\\
\{\pi_0 \lambda_0 - \pi_1 \mu_1 = 0,...,\pi_{n-1} \lambda_{n-1} - \pi_n \mu_n &=0, \sum_{i=1}^n \pi_i - 1 = 0\}
\end{align*}
$$

which gives is the matrix:

$$
\bcm \lambda_0 & -\mu_1 & 0 & 0 & ... & 0 & 0 & 0 & 0 \\
     0 & \lambda_1 & -\mu_2 & 0 & ... & 0 & 0 & 0 & 0 \\
     0 & 0 & \lambda_2 & -\mu_3 & ... & 0 & 0 & 0 & 0 \\
     ... & ... & ... & ... & ... & ... & ... & ... & ... \\
     0 & 0 & 0 & 0 & ... & \lambda_{n-3} & -\mu_{n-2} & 0 & 0 \\
     0 & 0 & 0 & 0 & ... & 0 & \lambda_{n-2} & -\mu_{n-1} & 0\\  
     0 & 0 & 0 & 0 & ... & 0 & 0 & \lambda_{n-1} & -\mu_n\\
     1 & 1 & 1 & 1 & ... & 1 & 1 & 1 & 1\ecm \bcm \pi_0\\
     \pi_1\\
     \pi_2\\
     ...\\
     \pi_{n-3}\\
     \pi_{n-2}\\
     \pi_{n-1}\\
     \pi_n\ecm = \bcm 0\\
     0\\
     0\\
     ...\\
     0\\
     0\\
     0\\
     1\ecm.
$$

Where, clearly, $\pi \neq 0$, as the last row would not equal 1. This has a solution then, thus $\pi$ is nontrivial.